In [ ]:
import network as layer
import torch.nn as nn

nc = 3
nz = 512

In [ ]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.utils.data
lr     = 0.0002
beta1  = 0.5      
imageSize = 64    
batchSize = 64    

outf = "./celeba_result/"
des_dir = "./celeba_crop/"

dataset = dset.ImageFolder(root=des_dir,
                           transform=transforms.Compose([
                               transforms.Resize(imageSize),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size= batchSize,
                                         shuffle=True)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        layers = []
        layers = layer.deconv(layers, nz, 512, 4, 1, 3, leaky=True, bn=False, wn=True, pixel=True)
        layers = layer.deconv(layers, 512, 512, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=True)
        
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers = layer.deconv(layers, 512, 256, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=True)
        layers = layer.deconv(layers, 256, 256, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=True)
        
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers = layer.deconv(layers, 256, 128, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=True)
        layers = layer.deconv(layers, 128, 128, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=True)
        
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers = layer.deconv(layers, 128, 64, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=True)
        layers = layer.deconv(layers, 64, 64, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=True)
        
        layers.append(nn.Upsample(scale_factor=2, mode='nearest'))
        layers = layer.deconv(layers, 64, 32, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=True)
        layers = layer.deconv(layers, 32, 32, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=True)
    
        layers = layer.deconv(layers, 32, nc, 1, 1, 0, leaky=True, bn=False, wn=True, pixel=True, only=True)
        self.network = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.network(x)
    
netG = Generator()

In [ ]:
from custom_layers import *
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        layers = []
        layers = layer.conv(layers, nc, 32, 1, 1, 0, leaky=True, bn=False, wn=True, pixel=False, gdrop=True)
        layers = layer.conv(layers, 32, 32, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True)
        layers = layer.conv(layers, 32, 64, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True)
        layers.append(nn.AvgPool2d(kernel_size=2))
        
        layers = layer.conv(layers, 64, 64, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True)
        layers = layer.conv(layers, 64, 128, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True)
        layers.append(nn.AvgPool2d(kernel_size=2))
        
        layers = layer.conv(layers, 128, 128, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True)
        layers = layer.conv(layers, 128, 256, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True)
        layers.append(nn.AvgPool2d(kernel_size=2))
        
        layers = layer.conv(layers, 256, 256, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True)
        layers = layer.conv(layers, 256, 512, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True)
        layers.append(nn.AvgPool2d(kernel_size=2))
        
        layers.append(minibatch_std_concat_layer())
        layers = layer.conv(layers, 513, 512, 3, 1, 1, leaky=True, bn=False, wn=True, pixel=False, gdrop=True)
        layers = layer.conv(layers, 512, 512, 4, 1, 0, leaky=True, bn=False, wn=True, pixel=False, gdrop=True)
        layers = layer.linear(layers, 512, 1, sig=False, wn=True)
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)
    
netD = Discriminator()

In [ ]:
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torchvision.utils as vutils

In [ ]:
criterion = nn.MSELoss()

input = torch.FloatTensor(batchSize, 3, imageSize,imageSize)
noise = torch.FloatTensor(batchSize, nz, 1, 1)
fixed_noise = torch.FloatTensor(batchSize, nz, 1, 1).normal_(0, 1)

label_real = torch.FloatTensor(batchSize)
label_fake = torch.FloatTensor(batchSize)

netD.cuda()
netG.cuda()
criterion.cuda()
input, label_real, label_fake = input.cuda(), label_real.cuda(), label_fake.cuda()
noise, fixed_noise = noise.cuda(), fixed_noise.cuda()
label_real.resize_(batchSize).fill_(1)
label_fake.resize_(batchSize).fill_(0)
label_real = Variable(label_real)
label_fake = Variable(label_fake)
print()

In [ ]:
fixed_noise = Variable(fixed_noise)

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

result_dict = {}
loss_D,loss_G,score_D,score_G1,score_G2 = [],[],[],[],[]

In [ ]:
niter = 100
for epoch in range(niter):
    for i, data in enumerate(dataloader, 0):

        # train D
        netD.zero_grad()
        real_cpu, _ = data
        batch_size = real_cpu.size(0)

        real_cpu = real_cpu.cuda()
        input.resize_as_(real_cpu).copy_(real_cpu)

        inputv = Variable(input)

        d_real = netD(inputv)
        d_real_mean = d_real.data.mean()
        
        noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        
        d_fake = netD(fake.detach())
        d_fake_mean = d_fake.data.mean()
        
        loss_d = criterion(d_real, label_real) + criterion(d_fake, label_fake)
        loss_d.backward()
        optimizerD.step()
        
        # train G
        netG.zero_grad()
        d_fake = netD(fake)
        loss_g = criterion(d_fake, label_real.detach())
        loss_g.backward()
        optimizerG.step()

        if i%250 == 0:
            fake = netG(fixed_noise)
            vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%03dstep_%04d.png' % (outf, epoch, i),
                    normalize=True)
            print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f'
                  % (epoch, niter, i, len(dataloader),
                     loss_d.data[0], loss_g.data[0], d_real_mean, d_fake_mean))